# RNA-seqカウントデータの前処理(1)

- サンプルごとのカウントデータを1つのカウントテーブルにまとめる

### 1. カウントテーブルの作成

kallistoではサンプル1個ごとのカウントデータが得られます<br>
ほかのツールで処理するときは全部のデータをまとめたほうが扱いやすいので、kallistoのカウント結果 `abundance.tsv` をまとめてひとつのカウントテーブルを作ります

必要なモジュールをインポートします

In [ ]:
import pandas as pd

SRAアクセッションを1行ずつ並べた `SRR_Acc_List.txt` を読み込みます

In [ ]:
# SRAアクセッションを1行ごとに並べたテキストファイルを読み込む
# 行末の改行コードを削除
sralib=[i[:-1] for i in open('data/SRR_Acc_List.txt','r')]
sralib = tuple(sralib) # 順番を変えたくないのでtupleにする
sralib

kallistoによるカウント結果　abundance.tsvのPATHのリストを作ります<br>
`data`フォルダの下の`kallisto`フォルダに、それぞれの結果フォルダが入っています

In [ ]:
kallisto_counts=[]

# SRAアクセッションを1つずつ読み込み、カウントデータのPATHを作成する
for sra in sralib:
    kallisto_counts.append('data/kallisto/' + sra + '_exp_kallisto/abundance.tsv')

kallisto_counts = tuple(kallisto_counts) # 順番を変えたくないのでtupleにする

確認します

In [ ]:
kallisto_counts

`abundance.tsv`を読み込んで、<br>
`estimate_count`列と`tpm`列にSRAアクセッションを追加する処理を、<br>
`read_countdata()`という関数にしておきます

In [ ]:
# kallisto結果ファイルのPATH（kallisto_count）のn番目(num = n)
def read_countdata(num):

	# kallisto結果ファイルの読み込み
    df = pd.read_table(kallisto_counts[num],sep='\t')

	# 'estimate_count'と'tpm'にSRAアクセッションを追加して新しい列名をつくる
    newcol1 = 'est_counts_' + sralib[num]
    newcol2 = 'tpm_' + sralib[num]

	# 新しい列名に更新
    df.rename(columns = {'est_counts':newcol1,'tpm':newcol2}, inplace=True)

    return df


カウントデータを読み込みます

In [ ]:
df0 = read_countdata(0)
df1 = read_countdata(1)
df2 = read_countdata(2)
df3 = read_countdata(3)
df4 = read_countdata(4)
df5 = read_countdata(5)

読み込めているか確認します（いずれも同じ行数になるはず）

In [ ]:
print(len(df0),len(df1),len(df3),len(df3),len(df4),len(df5))

df0の最初の5行を表示してみます

In [ ]:
df0.head()

`length`列, `eff_length`列（TPMの計算に使う）は最初に読み込んだ分だけでいいので、残りの表から削除します

In [ ]:
df1_count = df1.copy().drop(columns=['length','eff_length'])
df2_count = df2.copy().drop(columns=['length','eff_length'])
df3_count = df3.copy().drop(columns=['length','eff_length'])
df4_count = df4.copy().drop(columns=['length','eff_length'])
df5_count = df5.copy().drop(columns=['length','eff_length'])

target_idをkeyとして、すべてつなげます

In [ ]:
new_df = pd.merge(df0, df1_count, on = 'target_id')
new_df = pd.merge(new_df, df2_count, on = 'target_id')
new_df = pd.merge(new_df, df3_count, on = 'target_id')
new_df = pd.merge(new_df, df4_count, on = 'target_id')
new_df = pd.merge(new_df, df5_count, on = 'target_id')

確認します

In [ ]:
new_df.head()

`est_counts`と`eff_length` だけのtable、`tpm` だけのtableを作ります<br>
列名もSRAアクセッションのみにします

In [ ]:
est_countsとeff_lengthのテーブルを作ります

# 削除する列を指定
drop_column1 = ['tpm_'+ i for i in sralib]
drop_column1.append('length')

# 列名est_counts_SRRacc を　SRRaccに変換
# 順番と名前が変わったら困るので、辞書で指定する
rename_column1 = {'est_counts_'+ i:i for i in sralib}
new_df_count = new_df.copy().drop(columns = drop_column1).rename(columns=rename_column1)


確認します

In [ ]:
new_df_count.head()

tpmだけのテーブルを作成


In [ ]:
# 削除する列を指定
drop_column2 = ['est_counts_'+ i for i in sralib]
drop_column2.append('length')
drop_column2.append('eff_length')

# 列名'est_counts_SRRacc' を　'SRRacc'に変換
# 順番と名前が変わったら困るので、辞書で指定する
rename_column2 = {'tpm_'+ i:i for i in sralib}
new_df_tpm = new_df.copy().drop(columns = drop_column2).rename(columns=rename_column2)

確認します

In [ ]:
new_df_tpm.head()

タブ区切りファイルとして保存します

`pandas.DataFrame`のメソッド `pandas.DataFrame.to_csv()`を使い、タブ区切り`sep = '\t'`, インデックス列（上の例では0,1,2,3...）を非表示`index=False`にして書き出します

In [ ]:
new_df_count.to_csv('data/counts_kallisto.tsv', sep='\t',index=False)
new_df_tpm.to_csv('data/tpm_kallisto.tsv', sep='\t',index=False)

参考
- 実験医学別冊　独習Pythonバイオ情報解析　第6,7章　(2021年、先進ゲノム解析研究推進プラットフォーム編、ISBN978-4-7581-2249-8)
- pandas 公式サイト　https://pandas.pydata.org
- note.nknk.me pandas関連記事まとめ　https://note.nkmk.me/python-pandas-post-summary/
- kallisto を用いた A. thaliana paired-end リードの転写産物の定量 https://bi.biopapyrus.jp/rnaseq/mapping/kallisto/kallisto-paired.html
- Quasi-Mappingによって高速にRNA seqの定量を行う Kallisto https://kazumaxneo.hatenablog.com/entry/2018/07/14/180503
- kallisto Manual http://pachterlab.github.io/kallisto/manual.html
